In [92]:
import pandas_ta as pa
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime


def moving_average(financial_data: pd.DataFrame, record_to_plot: int = 2000, fig: go.Figure = None, dates=None):
    
    fig = go.Figure() if fig is None else fig
    

    # rename column from Close/Last to Close
    financial_data = financial_data[:record_to_plot]
    financial_data.rename(columns={"Close/Last": "Close"}, inplace=True)

    for date in dates:
        financial_data[f"MA{date}"] = pa.ema(financial_data.Close, length=date)

    # financial_data["MA7"] = pa.sma(financial_data.Close, length=7)
    # financial_data["MA15"] = pa.sma(financial_data.Close, length=15)
    # financial_data["MA21"] = pa.sma(financial_data.Close, length=21)

    # financial_data["MA60"] = pa.ema(financial_data.Close, length=60)
    # financial_data["MA120"] = pa.ema(financial_data.Close, length=120)
    financial_data.tail(10)

    def mysig(x, prev):
        # dates is a list of dates and we want to check if they cross the moving average
        dates.sort()
        shortest = dates[0]
        longest = dates[-1]

        if x[f"MA{shortest}"] > x[f"MA{longest}"] and prev == 1:
            prev = 2
            
            return 2, prev
        elif x[f"MA{shortest}"] < x[f"MA{longest}"] and prev == 2:
            prev = 1
           
            return 1, prev
        else:
            return 0, prev

    previous = 1
    for index, row in financial_data.iterrows():
        signal, previous = mysig(row, prev=previous)  # Call mysig for each row

        financial_data.loc[index, "signal"] = signal  # Assign signal to DataFrame
        if signal == 2:
            fig.add_annotation(
                x=index,
                y=row["Close"],
                text="✅",  # Unicode arrow pointing upwards as a buy signal
                showarrow=True,
                arrowhead=0,  # No arrowhead
                font=dict(size=20),  # Adjust font size as needed
            )
            
        elif signal == 1:
            fig.add_annotation(
                x=index,
                y=row["Close"],
                text="❌",  # Unicode arrow pointing downwards as a sell signal
                showarrow=True,
                arrowhead=0,  # No arrowhead
                font=dict(size=20),  # Adjust font size as needed
            )
    dfpl = financial_data[:record_to_plot]

    for date in dates:
        fig.add_trace(go.Scatter(
            x=dfpl.index, y=dfpl[f"MA{date}"], mode='lines', name=f'MA{date}'))
        # if signal is 1 then show a green arrow

    # how many signals were there
    print("There were", len(
        financial_data[financial_data["signal"] == 1]), "buy signals")
    print("There were", len(
        financial_data[financial_data["signal"] == -1]), "sell signals")

    # day one
    print("day one", financial_data.iloc[0].at_time)
    # day last
    print("day last", financial_data.iloc[-1].at_time)

    # days where the signal was 1 date
    buy_dates = []
    for index, row in financial_data.iterrows():
        if row.loc["signal"] == 2:
            date = row.values[0]
            position = financial_data.index.get_loc(index)
            buy_dates.append(position)
    print("days where the signal was 2", buy_dates)

    # days where the signal was -1 date
    sell_dates = []
    for index, row in financial_data.iterrows():
        if row.loc["signal"] == 1:
            date = row.values[0]
            position = financial_data.index.get_loc(index)
            sell_dates.append(position)
    print("days where the signal was 1", sell_dates)
    fig.show()
    return dfpl,financial_data

In [93]:
financial_data = pd.read_csv(
    "..\data\EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2021.csv")

# delete dollar sign
try:
    # Clean up dollar signs and convert columns to numeric values
    financial_data["Close/Last"] = financial_data["Close/Last"].str.replace(
        "$", "").astype(float)
    financial_data["Open"] = financial_data["Open"].str.replace(
        "$", "").astype(float)
    financial_data["High"] = financial_data["High"].str.replace(
        "$", "").astype(float)
    financial_data["Low"] = financial_data["Low"].str.replace(
        "$", "").astype(float)
except Exception:
    pass

# rename the headers
try:
    financial_data.rename(columns={"Close/Last": "Close"}, inplace=True)
except Exception:
    pass


In [94]:
financial_data.rename(columns={"high": "High"}, inplace=True)
financial_data.rename(columns={"low": "Low"}, inplace=True)
financial_data.rename(columns={"open": "Open"}, inplace=True)
financial_data.rename(columns={"volume": "Volume"}, inplace=True)
financial_data.rename(columns={"close": "Close"}, inplace=True)


financial_data = financial_data.reset_index(drop=True)

In [95]:
_, financial_data = moving_average(financial_data,record_to_plot=2000,dates=[4,15])


C:\Users\bobek\AppData\Local\Temp\ipykernel_15960\3411643274.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bobek\AppData\Local\Temp\ipykernel_15960\3411643274.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bobek\AppData\Local\Temp\ipykernel_15960\3411643274.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

There were 76 buy signals
There were 0 sell signals
day one <bound method NDFrame.at_time of date      04.05.2003 21:00:00.000
Open                      1.12354
High                      1.12354
Low                       1.12166
Close                     1.12274
Volume                 95533.0976
MA4                           NaN
MA15                          NaN
signal                        0.0
Name: 0, dtype: object>
day last <bound method NDFrame.at_time of date      11.08.2004 13:00:00.000
Open                       1.2221
High                      1.22285
Low                       1.21947
Close                     1.22196
Volume                323732.2989
MA4                      1.222303
MA15                     1.223079
signal                        0.0
Name: 1999, dtype: object>
days where the signal was 2 [14, 58, 81, 113, 143, 166, 245, 293, 322, 325, 329, 394, 412, 425, 439, 496, 505, 533, 588, 629, 635, 665, 688, 703, 720, 729, 809, 887, 955, 963, 993, 1004, 1022, 1047, 107

In [96]:
from apply_backtesting import backtester

bt,stats = backtester(financial_data)

c:\Users\bobek\QuantWiseAI\jupiter_notebooks\apply_backtesting.py:34: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [97]:
import backtesting

# import output_notebook
from bokeh.io import output_notebook
output_notebook()

backtesting.set_bokeh_output(notebook=True)
bt.plot()

Loading BokehJS ...

Row(id='6964', ...)

In [98]:
print(stats)

Start                                     0.0
End                                    1999.0
Duration                               1999.0
Exposure Time [%]                        97.9
Equity Final [$]                  10821.42982
Equity Peak [$]                   11135.93223
Return [%]                           8.214298
Buy & Hold Return [%]                8.837309
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -6.917198
Avg. Drawdown [%]                   -0.984373
Max. Drawdown Duration                 1047.0
Avg. Drawdown Duration              62.354839
# Trades                                 27.0
Win Rate [%]                        59.259259
Best Trade [%]                        4.12236
Worst Trade [%]                     -5.254051
Avg. Trade [%]                    